In [13]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [14]:
def normalize_date(text):
    # Biểu thức chính quy để bắt ngày, tháng, năm
    pattern = r"(\d{1,2})\s*tháng\s*(\d{1,2})\s*năm\s*(\d{4})"
    match = re.search(pattern, text)
    
    if match:
        # Lấy ra các thành phần ngày, tháng, năm
        day, month, year = match.groups()
        
        # Chuyển đổi thành định dạng dd-mm-yyyy
        date_obj = datetime.strptime(f"{day}-{month}-{year}", "%d-%m-%Y")
        return date_obj.strftime("%d-%m-%Y")
    return None

In [15]:
def part_text(part,start,end):
    part_list = []
    for i in part[start:end]:
        part_list.append(i.get_text().replace('\r', '').replace('\n', '').strip())
        part_text = '\n'.join(part_list)
    return part_text
        

In [16]:
def get_phanloai_link_info(link):
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')

    # Khởi tạo các biến trước để tránh lỗi khi không tìm thấy
    số = "Không tìm thấy 'Số'"
    ngày = "Không tìm thấy 'Ngày'"
    part_1_text = "Không tìm thấy part 1"
    part_2_text = "Không tìm thấy part 2"
    part_3_text = "Không tìm thấy part 3"
    part_4_text = "Không tìm thấy part 4"
    part_5_text = "Không tìm thấy thông tin phần 5"
    part_6_text = "Không tìm thấy thông tin phần 6" # Tổ chức thực thi

    # Lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'Số'
    except AttributeError as e:
        print(f"Lỗi khi tìm 'Số': {e}")

    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'ngày' in text or 'Ngày' in text:
                ngày = normalize_date(text)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
    except AttributeError as e:
        print(f"Lỗi khi tìm 'ngày': {e}")

    # Lấy part 1 - bảng đầu của trang
    part1 = tables[1].find_all('p')       
    part1_1_pattern = re.compile(r"1\s*\.\s*[Tt]ên\s*[Hh]àng")
    part1_2_pattern = re.compile(r"2\s*\.\s*Đơn\s*vị")
    part1_3_pattern = re.compile(r"3\s*\.\s*Số")
    part1_3_pattern_2 = re.compile(r"3\s*\.\s*Tờ\s*khai")
    part1_4_pattern = re.compile(r"4\s*\.\s*Tóm\s*tắt")
    part1_5_pattern = re.compile(r"5\s*\.\s*Kết\s*quả")

    # Tìm index các phần
    part1_1_index = next((i for i, p_tag in enumerate(part1) if part1_1_pattern.search(p_tag.get_text(strip=True))), None)
    part1_2_index = next((i for i, p_tag in enumerate(part1) if part1_2_pattern.search(p_tag.get_text(strip=True))), None)
    part1_3_index = next((i for i, p_tag in enumerate(part1) if part1_3_pattern.search(p_tag.get_text(strip=True)) or part1_3_pattern_2.search(p_tag.get_text(strip=True))), None)
    part1_4_index = next((i for i, p_tag in enumerate(part1) if part1_4_pattern.search(p_tag.get_text(strip=True))), None)
    part1_5_index = next((i for i, p_tag in enumerate(part1) if part1_5_pattern.search(p_tag.get_text(strip=True))), None)

    # Kiểm tra nếu các chỉ mục có tồn tại, nếu không gán giá trị thông báo không tìm thấy
    if part1_1_index is not None and part1_2_index is not None:
        part_1_text = part_text(part1, part1_1_index, part1_2_index)
    if part1_2_index is not None and part1_3_index is not None:
        part_2_text = part_text(part1, part1_2_index, part1_3_index)
    if part1_3_index is not None and part1_4_index is not None:
        part_3_text = part_text(part1, part1_3_index, part1_4_index)
    if part1_4_index is not None and part1_5_index is not None:
        part_4_text = part_text(part1, part1_4_index, part1_5_index)

    # Lấy thông tin phần 5 nếu tìm thấy index
    if part1_5_index is not None:
        pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
        for p_tag in part1[part1_5_index:]:
            text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
            matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
            if matches:
                part_5_text = matches[-1]  # Lấy kết quả cuối cùng khớp với pattern
                part_5_full = p_tag.get_text().replace('\r', '').replace('\n', '').strip()
                
    # Lấy thông tin phần 6
    for sibling in tables[1].previous_siblings:
        if sibling.name == 'p':
            p_before_table1 = sibling
            break
    # In ra thẻ <p> nếu tìm thấy
    if p_before_table1:
        part_6_text = p_before_table1.get_text().replace('\r', ' ').replace('\n', ' ').replace('  ', ' ').strip()

    # Tạo danh sách chứa thông tin
    info_list = [link.strip(), số, ngày, part_1_text, part_2_text, part_3_text, part_4_text, part_5_text,part_5_full,part_6_text]
    info_list = [re.sub(r'[ ]+', ' ', text).strip() for text in info_list]

    print(link)
    print(info_list)
    return info_list


In [17]:
with open('links_for_craw.txt', 'r', encoding='utf-8') as file:
    all_links = file.readlines()

In [18]:
len(all_links)

91

In [19]:
all_phanloai_info_list=[]
for link in all_links:
    if 'phan-loai' in link:
        try:
            info_list = get_phanloai_link_info(link)
            all_phanloai_info_list.append(info_list)
            # write_data_to_excel(phanloai_sheet,info_list)
        except:
            print(link + "lỗi")
            info_list = [link,"lỗi"]
            all_phanloai_info_list.append(info_list)
            # write_data_to_excel(phanloai_sheet,info_list)

In [20]:
df = pd.DataFrame(all_phanloai_info_list)

In [21]:
len(df)

0

In [ ]:
df.to_excel("phan_loai_18022025.xlsx",index = False)